<a href="https://colab.research.google.com/github/jefferson2601/Jogo_da_Memoria/blob/main/Jogo_Mem%C3%B3ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
pip install pygame


In [24]:
import pygame
import random
import os

In [25]:
pygame.init()

(4, 1)

In [26]:
# Configurações de tela e do jogo
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600
GRID_SIZE = 4  # Aumentamos o grid para 4x4 (16 cartas)
TILE_SIZE = SCREEN_WIDTH // GRID_SIZE
FPS = 30

In [27]:
# Cores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)


In [35]:
# Caminho das imagens (Você pode colocar suas próprias imagens aqui)
IMAGES_PATH = '/content/imagens/' # Adiciona uma barra no final do caminho
image_files = [f'{IMAGES_PATH}image{i}.png' for i in range(1, 9)]
image_files = image_files * 2  # Duplicando para ter pares de imagens

In [29]:
# Carregando as imagens
def load_images():
    images = []
    for img_file in image_files:
        img = pygame.image.load(img_file)
        img = pygame.transform.scale(img, (TILE_SIZE, TILE_SIZE))
        images.append(img)
    return images

In [37]:
# Função para desenhar o grid
def draw_grid(screen, cards, flipped_cards, images):
    screen.fill(WHITE)

    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            card_index = row * GRID_SIZE + col
            x = col * TILE_SIZE
            y = row * TILE_SIZE
            pygame.draw.rect(screen, GRAY, (x, y, TILE_SIZE, TILE_SIZE))  # Desenha o fundo da carta

            if card_index in flipped_cards:
                screen.blit(images[cards[card_index]], (x, y))  # Exibe a imagem da carta
            else:
                # Corrigido: adicionando largura e altura para a borda
                pygame.draw.rect(screen, BLACK, (x, y, TILE_SIZE, TILE_SIZE), 3)  # Borda das cartas

    pygame.display.flip()

In [31]:
# Função para embaralhar as cartas
def shuffle_board():
    cards = list(range(8)) * 2  # 8 pares de cartas
    random.shuffle(cards)
    return cards

In [32]:
# IA: A IA vai tentar "lembrar" as posições das cartas
def ai_turn(cards, flipped_cards, board_state):
    for i in range(len(cards)):
        if i not in flipped_cards:
            for j in range(i+1, len(cards)):
                if j not in flipped_cards and cards[i] == cards[j]:
                    return i, j  # IA tenta adivinhar os pares
    return None, None  # Caso não tenha encontrado nenhum par

In [33]:
# Função para verificar se o jogo foi resolvido
def check_win(board):
    count = 1
    for row in board:
        for tile in row:
            if tile != count % (GRID_SIZE * GRID_SIZE):
                return False
            count += 1
    return True

In [38]:
# Função principal do jogo
def main():
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Jogo da Memória")

    cards = shuffle_board()
    images = load_images()  # Carregando as imagens
    flipped_cards = []  # Lista de cartas viradas
    first_card = None  # A primeira carta virada
    second_card = None  # A segunda carta virada
    matched_pairs = 0  # Contador de pares encontrados
    board_state = {}  # Estado atual do jogo para a IA (posição das cartas)
    running = True
    player_turn = True  # Começa com o jogador
    clock = pygame.time.Clock()

    while running:
        draw_grid(screen, cards, flipped_cards, images)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN and player_turn:
                x, y = pygame.mouse.get_pos()
                col, row = x // TILE_SIZE, y // TILE_SIZE
                card_index = row * GRID_SIZE + col

                if card_index not in flipped_cards:
                    flipped_cards.append(card_index)

                    if first_card is None:
                        first_card = card_index
                    elif second_card is None:
                        second_card = card_index

                    if first_card is not None and second_card is not None:
                        # Checando se as cartas são iguais
                        if cards[first_card] == cards[second_card]:
                            matched_pairs += 1
                            first_card = None
                            second_card = None
                        else:
                            pygame.time.wait(500)  # Pausa para o jogador ver as cartas
                            flipped_cards.remove(first_card)
                            flipped_cards.remove(second_card)
                            first_card = None
                            second_card = None

                    player_turn = False  # Depois do jogador, é a vez da IA

        # A IA jogando
        if not player_turn:
            i, j = ai_turn(cards, flipped_cards, board_state)
            if i is not None and j is not None:
                flipped_cards.append(i)
                flipped_cards.append(j)
                if cards[i] == cards[j]:
                    matched_pairs += 1
                pygame.time.wait(1000)  # Pausa entre os turnos da IA
                player_turn = True  # Volta para o jogador

        if matched_pairs == len(cards) // 2:
            print("Jogo Finalizado!")
            running = False

        clock.tick(FPS)

    pygame.quit()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 